# PyTorch Neural-Transfert tutorial

## Introduction

Welcome! This tutorial explains how to impletment the [Neural-Style](https://arxiv.org/abs/1508.06576) algorithm developed by Leon A. Gatys, Alexander S. Ecker and Matthias Bethge.

### Neural what?

The Neural-Style, or Neural-Transfert, is an algorithm that taks as input a content-image (e.g. a tortle), a style-image (e.g. artistic waves) and return the content of the content-image as if it was 'painted' using the artistic style of the style-image:

![content1](images/neuralstyle.png)

### How does it work?

The principe is simple: we define two distances, one for the content ($D_C$) and one for the style ($D_S$). $D_C$ measues how different is the content between two images, while $D_S$ measures how different is the style between two images. Then, we take a third image, the input, (e.g. a with noise), and we transform it in order to both minimize its content-distance with the content-image and its style-distance with the style-image. 

### OK. How does it work?

Well, going further requires some mathematics. Let $C_{nn}$ be a pre-trained deep convolutional neural network and $X$ be any image. $C_{nn}(X)$ is the network fed by $X$ (containing feature maps at all layers). Let $F_{XL} \in C_{nn}(X)$ be the feature maps at depth layer $L$, all vectorized and concatenated in one single vector. We simply define the content of $X$ at layer $L$ by $F_{XL}$. Then, if $Y$ is another image of same the size than $X$, we define the distance of content at layer $L$ as follow:

$$D_C^L(X,Y) = \|F_{XL} - F_{YL}\|^2 = \sum_i (F_{XL}(i) - F_{YL}(i))^2$$

Where $F_{XL}(i)$ is the $i^{th}$ element of $F_{XL}$. The style is a bit less trivial to define. Let $F_{XL}^k$ with $k \leq K$ be the vectorized $k^{th}$ of the $K$ feature maps at layer $L$. The style $G_{XL}$ of $X$ at layer $L$ is defined by the Gram produce of all vectorized feature maps $F_{XL}^k$ with $k \leq K$. In other words, $G_{XL}$ is a $K$x$K$ matrix and the element $G_{XL}(k,l)$ at the $k^{th}$ line and $l^{th}$ column of $G_{XL}$ is the vectorial produce between  $F_{XL}^k$ and $F_{XL}^l$:

$$G_{XL}(k,l) = \langle F_{XL}^k, F_{XL}^l\rangle = \sum_i F_{XL}^k(i) . F_{XL}^l(i)$$

Where $F_{XL}^k(i)$ is the $i^{th}$ element of $F_{XL}^k$. We can see $G_{XL}(k,l)$ as a measure of the correlation between feature maps $k$ and $l$. In that way, $G_{XL}$ represents the correlation matrix of feature maps of $X$ at layer $L$. Note that the size of $G_{XL}$ only depends on the number of feature maps, not on the size of $X$. Then, if $Y$ is another image *of any size*, we define the distance of style at layer $L$ as follow:

$$D_S^L(X,Y) = \|G_{XL} - G_{YL}\|^2 = \sum_{k,l} (F_{XL}(k,l) - F_{YL}(k,l))^2$$

In order to minimize in one shot $D_C(X,C)$ between a variable image $X$ and target content-image $C$ and $D_S(X,S)$ between $X$ and target style-image $S$, both computed at several layers , we compute and sum the gradients (derivative with respect to $X$) of each distance at each wanted layer:

$$\nabla_{\textit{total}}(X,S,C) = \sum_{L_C} w_{CL_C}.\nabla_{\textit{content}}^{L_C}(X,C) + \sum_{L_S} w_{SL_S}.\nabla_{\textit{style}}^{L_S}(X,S)$$

Where $L_C$ and $L_S$ are respectivement the wanted layers (arbitrary stated) of content and style and $w_{CL_C}$ and $w_{SL_S}$ the weights (arbitrary stated) associated with the style or the content at each wanted layer. Then, we run a gradient descent over $X$:

$$X \leftarrow X - \alpha \nabla_{\textit{total}}(X,S,C)$$

Ok. That's enough with maths. If you want to go deeper (how to compute the gradients) **we encourage you to read the original paper** by Leon A. Gatys and AL, where everything is much better and much clearer explained. For our implementation in PyTorch, we already have everything we need: indeed, with PyTorch, all the gradients are automatically and dynamically computed for you (while you use functions from the library). This is why the implementation of this algorithm becomes very confortable with PyTorch. 

## PyTorch implementation

If you are not sure to understand all the mathematics above, you will probably get it by implementing it. If you are discovering PyTorch, we recommend you to first read this [Introduction to PyTorch](https://github.com/pytorch/tutorials/blob/master/Deep%20Learning%20with%20PyTorch.ipynb).

### Packages

We will have recourse to the following packages:

* `torch`, `torch.nn`, `numpy` (indispensables packages for neural networks with PyTorch)
* `torch.autograd.Variable` (dynamic computation of the gradient wrt a variable)
* `torch.optim` (efficient gradient descents)
* `PIL`, `PIL.Image`, `matplotlib.pyplot` (load and display images)
* `torchvision.transforms` (treat PIL images and transform into torch tensors)
* `torchvision.models` (train or load pre-trained models)

In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import PIL
from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

### Cuda

In this tutorial, we are assuming you have a GPU with cuda installed and that you want to use it. If you dont, just remove all the ''`.cuda()`'' and the ''`.cpu()`'' in the exemple codes. They are used to moove allocated proccesses associated with a variable (or a torch module) from the CPU to the GPU and vice versa.

### Load images

In order to simplify the implementation, let's start by importing a style and a content image of the same dimentions. We then scale them to the desired output image size (200 in the example) and transform them into torch tensors, ready to feed a neural network:

In [ ]:
imsize = 200 # desired size of the output image

loader = transforms.Compose([
            transforms.Scale(imsize), # scale imported image
            transforms.ToTensor()]) # transform it into a torch tensor

def image_loader(image_name):
    image = Image.open(image_name)
    image = Variable(loader(image))
    image = image.unsqueeze(0) # fake batch dimension required to fit network's input dimensions
    return image

style = image_loader("images/picasso.jpg").cuda()
content = image_loader("images/dancing.jpg").cuda()

assert style.data.size() == content.data.size(), "we need to import style and content images of the same size"

Imported PIL images has values between 0 and 255. Transformed into torch tensors, their values are between 0 and 1. This is an important detail: neural networks from torch library are trained with 0-1 tensor image. If you try to feed the networks with 0-255 tensor images the activated feature maps will have no sens. This is not the case with pre-trained networks from the Caffe library: they are trained with 0-255 tensor images.

### Display images

We will use `plt.imshow` to display images. So we need to first reconvert them into PIL images:

In [ ]:
unloader = transforms.ToPILImage() # reconvert into PIL image

def imshow(tensor):
    image = tensor.clone().cpu() # we clone the tensor in order to not do changes on it
    image.resize_(3,imsize,imsize) # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    
fig = plt.figure()

plt.subplot(221)
imshow(style.data)
plt.subplot(222)
imshow(content.data)

### Content loss

The content loss is a function that takes as input the feature maps $F_{XL}$ at a layer $L$ in a network fed by $X$ and return the weigthed content distance $w_{CL}.D_C^L(X,C)$ between this image and the content image. Hence, the weight $w_{CL}$ and the target content $F_{CL}$ are parameters of the function. We implement this function as a torch module with a constructor that takes these parameters as input. The distance $\|F_{XL} - F_{YL}\|^2$ is the Mean Square Error between the two sets of feature maps, that can be computed using a criterion `nn.MSELoss` stated as a third parameter.

We will add our content losses at each desired layer as additive modules of the neural network. That way, each time we will feed the network with an input image $X$, all the content losses will be computed at the desired layers and, thanks to autograd, all the gradients will be computed. For that, we just need to make the `forward` method of our module returning the input: the module becomes a ''transparent layer'' of the neural network. The computed loss is saved as a parameter of the module.

Finally, we define a fake `backward` method, that just call the backward method of `nn.MSELoss` in order to reconstruct the gradient. This method returns the computed loss: this will be usefull when running the gradien descent in order to display the evolution of style and content losses.

In [ ]:
class ContentLoss(nn.Module):
    def __init__(self, target, weight):
        super(ContentLoss, self).__init__()
        self.target = target.detach() * weight # we 'detach' the target content from the tree used
                                               # to dynamically compute the gradient: this is a stated value, 
                                               # not a variable. Otherwise the forward method of the criterion
                                               # will throw an error.
        self.weight = weight
        self.criterion = nn.MSELoss()

    def forward(self, input):
        self.loss = self.criterion.forward(input*self.weight, self.target)
        self.output = input
        return self.output

    def backward(self, retain_variables=True):
        self.loss.backward(retain_variables=retain_variables)
        return self.loss.data[0]

**Important detail**: this module, although it is named `ContentLoss`, is not a true PyTorch Loss function. If you want to define your content loss as a PyTorch Loss, you have to create a PyTorch autograd Function and to recompute/implement the gradient by the hand in the `backward` method. 

### Style loss

For the style loss, we need first to define a module that compute the gram produce $G_{XL}$ given the feature maps $F_{XL}$ of the neural network fed by $X$, at layer $L$. Let $\hat{F}_{XL}$ be the re-shaped version of $F_{XL}$ into a $K$x$N$ matrix, where $K$ is the number of feature maps at layer $L$ and $N$ the lenght of any vectorized feature map $F_{XL}^k$. The $k^{th}$ line of $\hat{F}_{XL}$ is $F_{XL}^k$. We let you check that $\hat{F}_{XL} \cdot \hat{F}_{XL}^T = G_{XL}$. Given that, it becomes easy to implement our module:

In [ ]:
class GramMatrix(nn.Module):
    def forward(self, input):
        a,b,c,d = input.data.size() # a=batch size(=1)
                                    # b=number of feature maps
                                    # (c,d)=dimensions of a f. map (N=c*d)
                
        input.data.resize_(a*b,c*d) # resise F_XL into \hat F_XL
        
        G = torch.mm(input, input.t()) # compute the gram product
        
        return G.div_(a*b*c*d) # we 'normalize' the values of the gram matrix
                           # by dividing by the number of element in each feature maps.

The longer is the feature maps dimension $N$, the bigger are the values of the gram matrix. Therefore, if we dont normalize by $N$, the loss computed at the first layers (before pooling layers) will have much more importance during the gradient descent. We dont want that, since the most interesting style features are in the deepest layers!

Then, the style loss module is implemented exactly the same way than the content loss module, but we have to add the `gramMatrix` as a parameter:

In [ ]:
class StyleLoss(nn.Module):
    def __init__(self, target, strength):
        super(StyleLoss, self).__init__()
        self.target = target.detach()*strength
        self.strength = strength
        self.gram = GramMatrix()
        self.criterion = nn.MSELoss()

    def forward(self, input):
        self.output = input.clone()
        self.G = self.gram.forward(input)
        self.G.mul_(self.strength)
        self.loss = self.criterion.forward(self.G, self.target)
        return self.output

    def backward(self, retain_variables=True):
        self.loss.backward(retain_variables=retain_variables)
        return self.loss.data[0]

### Load the neural network

Now, we have to import a pre-trained neural network. In the original paper, they obtain beautiful results using a re-trained VGG network with 19 layers. Unfortunatly, pre-trained VGG models are not available yet with the PyTorch library, but you can train your own VGG via [this example](https://github.com/pytorch/examples/tree/master/imagenet) or [load VGG caffe models](https://github.com/szagoruyko/loadcaffe). Right now, let's use a pre-trained AlexNet, available with PyTorch.

PyTorch's implementation of AlexNet is a module divided in two child `Sequential` modules: `features` (containing convolution and pooling layers) and `classifier` (containing fully connected layers). We are just interested by `features`:

In [ ]:
cnn = models.alexnet(pretrained=True).features.cuda()

A `Sequential` module contains an ordered list of child modules. For instance, `alexnet.features` contains a sequence (Conv2d, ReLU, Maxpool2d, Conv2d, ReLU...) aligned in the right order of depth. As we said in *Content loss* section, we wand to add our style and content loss modules as additive 'transparent' layers in our network, at desired depths. For that, we construct a new `Sequential` module, in wich we are going to add modules from `alexnet` and our loss modules in the right order:

In [ ]:
# desired depth layers to compute style/content losses :
content_layers = ['conv_3','conv_4']
style_layers = ['conv_1','conv_2','conv_3','conv_4','conv_5']

# just in order to have an iterable access to or list of content/syle losses
content_losses = []
style_losses = []

art_net = nn.Sequential().cuda() # the new Sequential module network
gram = GramMatrix().cuda() # we need a gram module in order to compute style targets

# weigth associated with content and style losses
content_weight = 5
style_weight = 500

i = 1
for layer in list(cnn):
    if isinstance(layer,nn.Conv2d):
        name = "conv_"+str(i)
        art_net.add_module(name,layer)

        if name in content_layers:
            # add content loss:
            target = art_net.forward(content.cuda()).clone()
            content_loss = ContentLoss(target, content_weight).cuda()
            art_net.add_module("content_loss_"+str(i),content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = art_net.forward(style.cuda()).clone()
            target_feature_gram = gram.forward(target_feature)
            style_loss = StyleLoss(target_feature_gram, style_weight).cuda()
            art_net.add_module("style_loss_"+str(i),style_loss)
            style_losses.append(style_loss)

    if isinstance(layer,nn.ReLU):
        name = "relu_"+str(i)
        art_net.add_module(name,layer)

        if name in content_layers:
            # add content loss:
            target = art_net.forward(content.cuda()).clone()
            content_loss = ContentLoss(target, content_weight).cuda()
            art_net.add_module("content_loss_"+str(i),content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = art_net.forward(style.cuda()).clone()
            target_feature_gram = gram.forward(target_feature)
            style_loss = StyleLoss(target_feature_gram, style_weight).cuda()
            art_net.add_module("style_loss_"+str(i),style_loss)
            style_losses.append(style_loss)

        i+=1

    if isinstance(layer,nn.MaxPool2d):
        name = "pool_"+str(i)
        art_net.add_module(name,layer) # ***


\*\*\* In the paper they recommend to change max pooling layers into average pooling. With AlexNet, that is a small network compared to VGG19 used in the paper, we are not going to see any difference of quality in the result. However, you can use these lines instead if you want to do this substitution:

In [ ]:
avgpool = nn.AvgPool2d(kernel_size=layer.kernel_size, stride=layer.stride, padding = layer.padding)
art_net.add_module(name,avgpool)

### Input image

Again, in order to simplify the code, we take an image of the same dimensions than content and style images. This image can be a white noise, or it can also be a copy of the content-image. 

In [ ]:
input = image_loader("images/dancing.jpg").cuda()
# if we want to fill it with a white noise:
input.data = torch.randn(input.data.size()).cuda()

# add the original input image to the figure:
plt.subplot(223)
imshow(input.data)

### Gradient descent

Let's use an Adam optimizer with a learning rate 0.01 to run our gradient descent. Unlike training a network, we want to train the input image in order to minimise the content/style losses. We would like to simply create a PyTorch Adam optimizer, passing our image as the variable to optimize. But `optim.Adam` takes as first argument a list of PyTorch `Variable` that require gradient. Our input image is a `Variable` but is not a leaf of the tree that requires computatioon of gradients. In order to show that this variable requires a gradient, a possibility is to construct a `Parameter` object from the input image. Then, we just give a list containing this `Parameter` to the optimizer's constructor:

In [ ]:
input = nn.Parameter(input.data) # this line to show that input is
                                 # a parameter that requires a gradient
optimizer = optim.Adam([input], lr = 0.01)

Last step: the loop of gradient descent. At each step, we must feed the network with the updated input in order to compute the new losses, we must run the `backward` methods of each loss to dynamically compute their gradients and perform the step of gradient descent:

In [ ]:
# -- WRONG CODE --

for run in range(1000):
    optimizer.zero_grad()
    art_net.forward(input) # feed the network with the updated input image
    style_score = 0
    content_score = 0

    for sl in style_losses:
        style_score += sl.backward() # run style losses backward
    for cl in content_losses:
        content_score += cl.backward() # run content losses backward

    optimizer.step() # update input image with descent's step

    if run%50==0:
        print "run "+str(run)+":"
        print "style loss: "+str(style_score)
        print "content loss: "+str(content_score)
        
# finally enjoy the result:
plt.subplot(224)
imshow(input.data)
plt.show()

If you try the code just above, you will obtain a beautiful white noise! Indeed, there is a last trick: the resulting optimized image takes its values between $-\infty$ and $+\infty$ instead of staying between 0 and 1. In other words, the image is well optimized, but has absurd values. In fact, we must perform an optimization under constraints in order to keep having right vaues into our input image. There is a simple solution: at each step, to correct the image to maintain its values into the 0-1 interval. 

In [ ]:
# -- CORRECT CODE --

for run in range(500):

    # correct the values of updated input image
    updated = input.data.cpu().clone()
    updated = updated.numpy()
    updated[updated<0] = 0
    updated[updated>1] = 1
    input.data = torch.from_numpy(updated).cuda()

    optimizer.zero_grad()
    art_net.forward(input.image)
    style_score = 0
    content_score = 0

    for sl in style_losses:
        style_score += sl.backward()
    for cl in content_losses:
        content_score += cl.backward()

    optimizer.step()

    if run%10==0:
        print "run "+str(run)+":"
        print style_score
        print content_score

# a last correction...
result = input.data.cpu().clone()
result = result.numpy()
result[result<0] = 0
result[result>1] = 1
result = torch.from_numpy(result)

# finally enjoy the result:
plt.subplot(224)
imshow(input.data)
plt.show()

### Results

You should obtain something similar to this:
![result](images/result_random.png)
You are probably desapointed by this result. Ok, we can do a bit better by starting from the content image instead of a white noise:
![result](images/result_content.png)
But this is normal to obtain something creepy: we are using a small network with just 5 convolution layers. In the paper, they obtain beautifull images using the 19 convolution layers of VGG19. If you want, we encourage you to test this code with a VGG19 network and play with parameters: style/content layer depths, style/content weights, learning rate, input (random or content-image)... 

With exactly the same code but using a pre-trained VGG19 network and playing with parameters, we obtained this result:

![result](images/vgg19_result.png)